# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [14]:
city_data_df.info


<bound method DataFrame.info of      City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0          0         faya  18.3851   42.4509     22.06        35          21   
1          1      farsund  58.0948    6.8047     13.30       100           0   
2          2  new norfolk -42.7826  147.0587     11.72        58          12   
3          3    jamestown  42.0970  -79.2353      5.77        77         100   
4          4      lanzhou  36.0564  103.7922     14.53        48          59   
..       ...          ...      ...       ...       ...       ...         ...   
572      572    sovetskiy  61.3614   63.5842      3.93        96         100   
573      573      tatarsk  55.2190   75.9828      3.78        89          84   
574      574    canchungo  12.0672  -16.0333     35.07        61          67   
575      575      iquique -20.2208  -70.1431     17.02        68           6   
576      576         poum -20.2333  164.0167     23.91        79          23   

     Wi

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale= 1.0
      )

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_city_df = city_data_df[(city_data_df["Max Temp"] <=70) & (city_data_df["Humidity"] <= 50) & (city_data_df["Cloudiness"] >=20.0)]


# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
22,22,guiratinga,-16.3494,-53.7583,38.35,14,22,3.10,BR,1666108243
30,30,midwest city,35.4495,-97.3967,9.12,33,75,6.17,US,1666108247
36,36,drochia,48.0350,27.8161,13.39,30,73,3.46,MD,1666108253


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Lat","Lng","Country","Humidity"]].copy()

hotel_df["Hotel Name"] = ""
hotel_df.head()




,City,Lat,Lng,Country,Humidity,Hotel Name
0,faya,18.3851,42.4509,SA,35,
4,lanzhou,36.0564,103.7922,CN,48,
22,guiratinga,-16.3494,-53.7583,BR,14,
30,midwest city,35.4495,-97.3967,US,33,
36,drochia,48.0350,27.8161,MD,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accomodation.hotel", "apiKey": geoapify_key, "limit": 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index,'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
lanzhou - nearest hotel: No hotel found
guiratinga - nearest hotel: No hotel found
midwest city - nearest hotel: No hotel found
drochia - nearest hotel: No hotel found
rafaela - nearest hotel: No hotel found
arman - nearest hotel: No hotel found
rio gallegos - nearest hotel: No hotel found
siavonga - nearest hotel: No hotel found
nova olimpia - nearest hotel: No hotel found
palencia - nearest hotel: No hotel found
yidu - nearest hotel: No hotel found
baruun-urt - nearest hotel: No hotel found
sao miguel do araguaia - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
ouallam - nearest hotel: No hotel found
neepawa - nearest hotel: No hotel found
hovd - nearest hotel: No hotel found
shakawe - nearest hotel: No hotel found
upington - nearest hotel: No hotel found
gwanda - nearest hotel: No hotel found
cabra - nearest hotel: No hotel found
asau - nearest hotel: No hotel found

,City,Lat,Lng,Country,Humidity,Hotel Name
0,faya,18.3851,42.4509,SA,35,No hotel found
4,lanzhou,36.0564,103.7922,CN,48,No hotel found
22,guiratinga,-16.3494,-53.7583,BR,14,No hotel found
30,midwest city,35.4495,-97.3967,US,33,No hotel found
36,drochia,48.0350,27.8161,MD,30,No hotel found
86,rafaela,-31.2503,-61.4867,AR,42,No hotel found
92,arman,59.7000,150.1667,RU,49,No hotel found
102,rio gallegos,-51.6226,-69.2181,AR,26,No hotel found
103,siavonga,-16.5382,28.7088,ZM,32,No hotel found
105,nova olimpia,-14.7972,-57.2881,BR,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Hotel Name"],
    geo = True,
    tiles = "OSM",
    color = 'Country',
    frame_width = 700,
    frame_height = 400
)


# Display the map
map_plot_2
    

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Hotel Name)